# Cell types analysis

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7335069/

## Read in BMI data only for the left hemi (because AHBA)

In [2]:
import nibabel as nib
from neuromaps.datasets import available_annotations
from neuromaps import datasets
from neuromaps.resampling import resample_images
from neuromaps import parcellate
from nilearn.datasets import fetch_atlas_surf_destrieux
from neuromaps import datasets, images, nulls, resampling, plotting
from neuromaps.datasets import fetch_annotation
from neuromaps import transforms
from neuromaps.stats import compare_images
#from brainspace.plotting import plot_surf
import numpy as np
from nibabel import freesurfer

# Read the basic files

BMI_HCP=list(['/dagher/dagher11/filip/Obesity_maps/data/BMI_lh.gii'])

BMI_ctx=transforms.fsaverage_to_fsaverage(BMI_HCP, '164k', hemi='L')

# Read mask of medial wall for fsaverage

medwall_idx_fsaverage_lh=(freesurfer.io.read_label('/dagher/dagher10/yyau/toolbox/freesurfer/subjects/fsaverage/label/lh.Medial_wall.label',
                        read_scalars=False))


# Set all vertices of medial wall to nan

BMI_ctx[0].agg_data()[medwall_idx_fsaverage_lh] = np.nan
BMI=BMI_ctx[0].agg_data()

/dagher/dagher12/filip/software/miniconda3py/envs/py3/lib/python3.10/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


## Read in Cammoun 125 template annot file for lh (because AHBA has data for lh mostly)

In [284]:
Cammoun125=freesurfer.io.read_annot('/home/bic/fmorys/nnt-data/atl-cammoun2012/fsaverage/atl-Cammoun2012_space-fsaverage_res-125_hemi-L_deterministic.annot')[0]

## Manually parcellate the data and remove 'unknown' and 'corpus callosum'

In [285]:
BMI_Cammoun125=list()

for i in range(113):
    BMI_Cammoun125.append(np.average(BMI[Cammoun125==i]))
    
BMI_Cammoun125.pop(0)
BMI_Cammoun125.pop(3)

nan

# Use cell-type data from Christina - no correlation between DBM maps and cell types using Cammoun 125

In [5]:
import mat73
import scipy
mat1 = mat73.loadmat('/dagher/dagher11/filip/Obesity_maps/data/GeneExpression_Cammoun234R_Astro_Ex.mat')
mat2 = scipy.io.loadmat('/dagher/dagher11/filip/Obesity_maps/data/GeneExpression_Cammoun234R_Endo_Ex.mat')
mat3 = mat73.loadmat('/dagher/dagher11/filip/Obesity_maps/data/GeneExpression_Cammoun234R_Micro_Ex.mat')
mat4 = mat73.loadmat('/dagher/dagher11/filip/Obesity_maps/data/GeneExpression_Cammoun234R_NeuroEx_Ex.mat')
mat5 = mat73.loadmat('/dagher/dagher11/filip/Obesity_maps/data/GeneExpression_Cammoun234R_NeuroIn_Ex.mat')
mat6 = mat73.loadmat('/dagher/dagher11/filip/Obesity_maps/data/GeneExpression_Cammoun234R_OPC_Ex.mat')
mat7 = scipy.io.loadmat('/dagher/dagher11/filip/Obesity_maps/data/GeneExpression_Cammoun234R_Oligo_Ex.mat')

## Read in Euclidean distance matrix from Christina

In [6]:
Euc_125_L=pd.read_csv('/dagher/dagher11/filip/Downloads/Euc_dist125_L_brainstem.txt', sep=',',header=None)

## Get surrogate maps for BMI only L hemisphere

In [9]:
import pandas as pd
import numpy as np
from brainsmash.mapgen.base import Base

brain_map_file = np.array(BMI_Cammoun125)
dist_mat_file = Euc_125_L.values[0:111,0:111] #remove subcortical structures from the Euclidean distances file
base = Base(x=brain_map_file, D=dist_mat_file)

surrogates = base(n=1000)
spin1000=pd.DataFrame(data=surrogates)
spin1000.to_csv("/dagher/dagher9/filip/BMIarea_abagen_L_Cammoun125_1000spin_smash.csv",sep=',',index=False)

NameError: name 'BMI_Cammoun125' is not defined

## Run correlations

In [45]:
import numpy.ma as ma

main_corr=ma.corrcoef(ma.masked_invalid(BMI_parc[0,115:234]), ma.masked_invalid(mat2['Mean_Gene'][115:234,0]))[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(ma.corrcoef(surrogates[spin], mat2['Mean_Gene'][115:234,0])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=ma.corrcoef(ma.masked_invalid(BMI_parc[0,115:234]), ma.masked_invalid(mat7['Mean_Gene'][115:234,0]))[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(ma.corrcoef(surrogates[spin], mat7['Mean_Gene'][115:234,0])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=ma.corrcoef(ma.masked_invalid(BMI_parc[0,115:234]), ma.masked_invalid(mat1['Mean_Gene'][115:234]))[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(ma.corrcoef(surrogates[spin], mat1['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=ma.corrcoef(ma.masked_invalid(BMI_parc[0,115:234]), ma.masked_invalid(mat3['Mean_Gene'][115:234]))[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(ma.corrcoef(surrogates[spin], mat3['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=ma.corrcoef(ma.masked_invalid(BMI_parc[0,115:234]), ma.masked_invalid(mat4['Mean_Gene'][115:234]))[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(ma.corrcoef(surrogates[spin], mat4['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=ma.corrcoef(ma.masked_invalid(BMI_parc[0,115:234]), ma.masked_invalid(mat5['Mean_Gene'][115:234]))[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(ma.corrcoef(surrogates[spin], mat5['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=ma.corrcoef(ma.masked_invalid(BMI_parc[0,115:234]), ma.masked_invalid(mat6['Mean_Gene'][115:234]))[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(ma.corrcoef(surrogates[spin], mat6['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

0.14862201835329658
0.4645354645354645
0.060750364323468295
0.5384615384615384
-0.02483766368092532
0.9400599400599401
-0.09314304147171178
0.7632367632367633
0.174269901843664
0.32667332667332666
0.10496784130589532
0.5694305694305695
-0.10616164475033932
0.7422577422577422


# Same for DBM

In [153]:
from netneurotools import datasets as nntdatasets
from neuromaps import datasets as nmdatasets
from neuromaps.parcellate import Parcellater
import pandas as pd
from neuromaps import transforms
import neuromaps
import numpy as np



cammoun = nntdatasets.fetch_cammoun2012(version="MNI152NLin2009aSym")
info = pd.read_csv(cammoun['info'])
labels = info.query('scale=="scale125"')['label']  # or might be "labels"

parc = Parcellater(cammoun['scale125'], 'mni152')  # assuming mymap is in mni152 space
BMI = neuromaps.images.load_nifti('/dagher/dagher9/filip/DBM/HCP_DBM_Slvl/beta_0003.nii')

mymap_parc = parc.fit_transform(BMI, 'mni152')

## Create surrogate maps with subcortical structures

In [89]:
import pandas as pd
from brainsmash.mapgen.base import Base

brain_map_file = mymap_parc[0,115:234]
dist_mat_file = Euc_125_L.values
base = Base(x=brain_map_file, D=dist_mat_file)

surrogates = base(n=10000)
spin1000=pd.DataFrame(data=surrogates)
spin1000.to_csv("/dagher/dagher9/filip/BMI_abagen_L_Cammoun125_10000spin_smash.csv",sep=',',index=False)

## Run correlations

In [181]:

main_corr=np.corrcoef(mymap_parc[0, 115:234], mat2['Mean_Gene'][115:234,0])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], mat2['Mean_Gene'][115:234,0])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=np.corrcoef(mymap_parc[0, 115:234], mat7['Mean_Gene'][115:234,0])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], mat7['Mean_Gene'][115:234,0])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)
        
main_corr=np.corrcoef(mymap_parc[0, 115:234], mat1['Mean_Gene'][115:234])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], mat1['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=np.corrcoef(mymap_parc[0, 115:234], mat3['Mean_Gene'][115:234])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], mat3['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=np.corrcoef(mymap_parc[0, 115:234], mat4['Mean_Gene'][115:234])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], mat4['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=np.corrcoef(mymap_parc[0, 115:234], mat5['Mean_Gene'][115:234])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], mat5['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

main_corr=np.corrcoef(mymap_parc[0, 115:234], mat6['Mean_Gene'][115:234])[1,0]
print(main_corr)
corrs=list()
for spin in range(1000):
    corrs.append(np.corrcoef(surrogates[spin], mat6['Mean_Gene'][115:234])[1,0])
print(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)

0.14862201835329655
0.4595404595404595
0.0607503643234683
0.5584415584415584
-0.02483766368092532
0.9400599400599401
-0.09314304147171176
0.7822177822177823
0.17426990184366403
0.31868131868131866
0.1049678413058953
0.5404595404595405
-0.10616164475033932
0.7632367632367633


# Gene analysis

## Get genetics data

In [1]:
import abagen
atlas = abagen.fetch_desikan_killiany()

In [2]:
files = abagen.fetch_microarray(data_dir='/dagher/dagher9/filip/', donors='all', verbose=0)

In [3]:
expression_125 = abagen.get_expression_data('/home/bic/fmorys/nnt-data/atl-cammoun2012/MNI152NLin2009aSym/atl-Cammoun2012_space-MNI152NLin2009aSym_res-125_deterministic.nii.gz',
                                            '/dagher/dagher11/filip/Obesity_maps/data/Cammoun_125_all.csv', 
                                            data_dir='dagher/dagher9/filip/',
                                            missing='interpolate')

/dagher/dagher12/filip/software/miniconda3py/envs/py3/lib/python3.10/site-packages/abagen/probes_.py:66: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  reannotated.loc[:, 'entrez_id'] = reannotated['entrez_id'].astype('int')
/dagher/dagher12/filip/software/miniconda3py/envs/py3/lib/python3.10/site-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/dagher/dagher12/filip/software/miniconda3py/envs/py3/lib/python3.10/site-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_ax

## Use Cammoun 125 parcellation for BMI data

In [4]:
from netneurotools import datasets as nntdatasets
from neuromaps import datasets as nmdatasets
from neuromaps.parcellate import Parcellater
import pandas as pd

cammoun = nntdatasets.fetch_cammoun2012(version="MNI152NLin2009aSym")
info = pd.read_csv(cammoun['info'])
labels = info.query('scale=="scale125"')['label']  

parc = Parcellater(cammoun['scale125'], 'mni152', hemi='L')  
mymap_parc = parc.fit_transform('/dagher/dagher9/filip/DBM/HCP_DBM_Slvl/beta_0003.nii', 'mni152')

/dagher/dagher12/filip/software/miniconda3py/envs/py3/lib/python3.10/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


## Add labels to expression data

In [11]:
expression1=expression_125
expression1['struct']=labels.values
expression1=expression1.reset_index()
expression1=expression1.set_index('struct')
expression1=expression1[0:108]

In [28]:
import pandas
import scipy

expr_np=pandas.DataFrame.to_numpy(expression1)
scipy.io.savemat('/dagher/dagher11/filip/Obesity_maps/data/Cammoun_aba_RH_thickness.mat', 
                 {'Cammoun_aba_RH_thickness': expr_np})

In [12]:
expression1.to_csv('/dagher/dagher11/filip/Obesity_maps/data/abagenes_Cammoun125_RH.csv', sep='\t')

## Correlate using previous surrogate maps for DBM; do it for specific genes 

## GFAP MAPT TNF IL-6 CCL2 CCR2 CLDN5, TJP1, ABCB1, APOE, NSE

In [65]:
gene_corrs=pd.DataFrame([[0, 0, 0]], columns=['gene','corrcoef','p-val'], index=['gene'])
index=0

for gene in ['GFAP','MAPT','TNF','CCL2','CLDN5','TJP1','ABCB1','APOE']:
        main_corr=np.corrcoef(mymap_parc[0,115:234], expression1[gene])[1,0]
        print('Gene number '+str(index))
        index=index+1
        #print(main_corr)
        corrs=list()
        for spin in range(1000):
            corrs.append(np.corrcoef(surrogates[spin], expression1[gene])[1,0])
            p=(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)
        gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'], 
                                                  index=[gene]))
        
gene_corrs.to_csv(path_or_buf='/dagher/dagher11/filip/Obesity_maps/data/gene_maps_correlations_10000.csv', sep=',',
                 header=True)

print(gene_corrs)

Gene number 0


/tmp/ipykernel_30535/13276042.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


Gene number 1


/tmp/ipykernel_30535/13276042.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


Gene number 2


...

/tmp/ipykernel_30535/13276042.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


Gene number 6


/tmp/ipykernel_30535/13276042.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


Gene number 7
        gene  corrcoef     p-val
gene       0  0.000000  0.000000
GFAP    GFAP  0.054389  0.867133
MAPT    MAPT  0.099991  0.619381
TNF      TNF -0.210950  0.543457
CCL2    CCL2 -0.083087  0.777223
CLDN5  CLDN5  0.078188  0.555445
TJP1    TJP1 -0.034389  0.831169
ABCB1  ABCB1 -0.023548  0.822178
APOE    APOE -0.065746  0.843157


/tmp/ipykernel_30535/13276042.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


In [90]:
gene_corrs=pd.DataFrame([[0, 0, 0]], columns=['gene','corrcoef','p-val'], index=['gene'])
index=0

for gene in expression1:
        main_corr=np.corrcoef(mymap_parc[0,115:234], expression1[gene])[1,0]
        print('Gene number '+str(index))
        index=index+1
        #print(main_corr)
        corrs=list()
        for spin in range(10000):
            corrs.append(np.corrcoef(surrogates[spin], expression1[gene])[1,0])
            p=(sum(i > abs(main_corr) for i in np.absolute(corrs))/10001)
        gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'], 
                                                  index=[gene]))
        
gene_corrs.to_csv(path_or_buf='/dagher/dagher11/filip/Obesity_maps/data/gene_maps_correlations_10000.csv', sep=',',
                 header=True)

Gene number 0


KeyboardInterrupt: 

In [212]:
from statsmodels.stats import multitest

corrected=multitest.multipletests(gene_corrs['p-val'], alpha=0.05, method='fdr_bh')

In [213]:
gene_corrs['cor_p']=corrected[1]

In [220]:
for gene in (gene_corrs[gene_corrs['p-val']<0.05].index.values):
    print(gene)

gene
ABCF3
ABCG1
ABHD16A
ABHD18
ABRAXAS2
ACADSB
ACAP2
ACAP3
ACAT2
ACKR4
ACSF3
ACSM1
ACTN2
ADA2
ADAMTSL2
ADCY8
ADCYAP1
ADGRG5
ADPRHL1
ADRA1B
ADRM1
ADTRP
AGBL3
AGFG1
AGL
AGTPBP1
AKAP11
AKAP12
AKAP8
ALAS1
ALDH1L2
ALDH5A1
ALDOA
ALKBH3
ALKBH5
ALMS1
ALMS1P1
ALPP
ALS2CL
ALS2CR12
ANKFY1
ANKRA2
ANKRD17
ANKRD27
ANKRD28
ANKRD30B
ANKRD36B
ANKRD36BP2
ANKRD44
ANKRD45
ANKS1B
ANP32D
AP1M1
AP2S1
AP4B1
AP4M1
APCDD1L
APEH
APLP2
APOL5
AQP2
ARFIP2
ARHGAP20
ARHGAP25
ARHGDIG
ARHGEF33
ARID3B
ARL4D
ARL5A
ARL8B
ARSK
ARTN
ASB16
ASB18
ASB2
ASB6
ASGR2
ASNSP1
ATAD2B
ATE1
ATG101
ATG2A
ATG5
ATG9A
ATP10D
ATP13A2
ATP2C1
ATP5PF
ATP6V0A1
ATP6V1A
ATRX
ATXN3
AURKA
AURKAIP1
AURKC
AUTS2
AVP
AZIN2
BAALC
BAIAP2L2
BANP
BCAS3
BCL2L11
BCL2L13
BEND5
BEX4
BHLHE22
BIN3-IT1
BIRC2
BNIP1
BNIP3
BOD1
BRD2
BTBD1
BTBD10
BTF3
BTG1
BTN2A3P
BTRC
C11orf49
C11orf74
C12orf29
C14orf28
C19orf44
C20orf144
C20orf96
C22orf15
C2orf49
C2orf70
C3orf38
C5orf22
C8orf46
C9orf139
CAAP1
CABLES1
CACNA1E
CACNB2
CALHM1
CALM3
CALN1
CAMK1G
CAPNS1
CAPRIN2
CARF
CAS

# Take genes significantly related to BMI according to 2018 GWAS and get an average map of distribution in the brain

## From Yengo

In [91]:
import itertools
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

Yengo = pd.read_csv('/dagher/dagher11/filip/Obesity_maps//data/Yengo_genes.csv')

common_genes=intersection(list(expression1.columns), list(itertools.chain(*Yengo.values.tolist())))

In [92]:
len(common_genes)

429

In [93]:
gene_maps_BMI=expression1[common_genes]
#gene_maps_BMI['mean']=gene_maps_BMI.mean(axis=1)

In [96]:
gene_maps_BMI.to_csv(path_or_buf='/dagher/dagher11/filip/Obesity_maps/data/genes_Yengo_LH.csv', sep=',',
                 header=True)

### Correlate BMI map with gene map

In [83]:
import mne

In [79]:
gene_corrs=pd.DataFrame([[0, 0, 0]], columns=['gene','corrcoef','p-val'], index=['gene'])
index=0

for gene in gene_maps_BMI:    
    main_corr=np.corrcoef(mymap_parc[0,115:234], gene_maps_BMI[gene])[1,0]
    print(main_corr)
    corrs=list()
    for spin in range(1000):
        corrs.append(np.corrcoef(surrogates[spin], gene_maps_BMI[gene])[1,0])
    p=(sum(i > abs(main_corr) for i in np.absolute(corrs))/1001)
    gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'], 
                                                  index=[gene]))

-0.03383940525207766
-0.18912192398020447


/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],
/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


-0.010368820553948414
0.41641519110804387


/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],
/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


0.1854253282789962
0.029909935704495245


...

/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],
/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],
/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


-0.2518392962985493
-0.11749544959045358


/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],
/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


0.07943551323107982


/tmp/ipykernel_30535/3929790965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gene_corrs=gene_corrs.append(pd.DataFrame([[gene, main_corr, p]],columns=['gene','corrcoef','p-val'],


In [90]:
(gene_corrs['p-val'])

gene        0.000000
A3GALT2     0.767233
ABCD2       0.572428
ABCG2       0.937063
ADAMTSL3    0.111888
              ...   
ZNF821      0.325674
ZSCAN31     0.919081
ZSWIM5      0.208791
ZSWIM6      0.592408
mean        0.721279
Name: p-val, Length: 431, dtype: float64

In [85]:
mne.stats.fdr_correction(gene_corrs['p-val'])

(array([ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
      

## Plot genes on surface

## Reverse parcellate data - remove unknown + CC

In [107]:
from nibabel import freesurfer

img_annot = freesurfer.read_annot('/home/bic/fmorys/nnt-data/atl-cammoun2012/fsaverage/atl-Cammoun2012_space-fsaverage_res-125_hemi-L_deterministic.annot')
BMI_genes_voxelwise=img_annot[0]

In [113]:
BMI_mean_genes=gene_maps_BMI['mean'][0:111]

In [114]:
dictionary_labels=pd.DataFrame(img_annot[2], np.unique(img_annot[0]))

In [115]:
dictionary_labels.index.name = 'label'
dictionary_labels=dictionary_labels.drop(0)
dictionary_labels=dictionary_labels.drop(4)

In [116]:
dictionary_labels

,0
label,
1,b'lateralorbitofrontal_1'
2,b'lateralorbitofrontal_2'
3,b'lateralorbitofrontal_3'
5,b'lateralorbitofrontal_4'
6,b'parsorbitalis_1'
...,...
108,b'transversetemporal_1'
109,b'insula_1'
110,b'insula_2'


In [117]:
BMI_label=pd.DataFrame(BMI_mean_genes.values, dictionary_labels.index)

In [118]:
for i in range(1,113):
    if i==4:
        BMI_genes_voxelwise[np.where(BMI_genes_voxelwise == i)]=0
        continue
    BMI_genes_voxelwise[np.where(BMI_genes_voxelwise == i)]=BMI_label.loc[i]*100000

In [119]:
np.savetxt('/dagher/dagher11/filip/Obesity_maps/data/GWAS_genes_map_Yengo.csv', BMI_genes_voxelwise, delimiter=',')

## For volumetric atlas

In [128]:
import nibabel as nib

img = nib.load(cammoun['scale125'])
BMI_genes_parcel=img

In [129]:
BMI_mean_genes=gene_maps_BMI['mean']

In [130]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(gene_maps_BMI['mean'])

116    0.515078
117    0.521666
118    0.530486
119    0.525705
120    0.519592
121    0.516944
122    0.528438
123    0.518535
124    0.534659
125    0.529556
126    0.532756
127    0.533203
128    0.517177
129    0.527652
130    0.525446
131    0.504273
132    0.527186
133    0.520097
134    0.526609
135    0.528011
136    0.543615
137    0.516398
138    0.521448
139    0.515942
140    0.515705
141    0.521306
142    0.521279
143    0.516519
144    0.525284
145    0.521144
146    0.521205
147    0.528464
148    0.514055
149    0.526998
150    0.520155
151    0.528963
152    0.529598
153    0.522455
154    0.536473
155    0.518327
156    0.496834
157    0.523340
158    0.530753
159    0.519345
160    0.509513
161    0.531613
162    0.536477
163    0.530056
164    0.510602
165    0.530945
166    0.527429
167    0.521861
168    0.506758
169    0.528246
170    0.520607
171    0.529436
172    0.516561
173    0.528475
174    0.529283
175    0.534411
176    0.525474
177    0.529171
178    0

In [131]:
labels.index = range(1,235)
labels_BMI=labels[115:len(labels)]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(labels_BMI)

116        lateralorbitofrontal_1
117        lateralorbitofrontal_2
118        lateralorbitofrontal_3
119        lateralorbitofrontal_4
120               parsorbitalis_1
121                 frontalpole_1
122         medialorbitofrontal_1
123         medialorbitofrontal_2
124            parstriangularis_1
125             parsopercularis_1
126             parsopercularis_2
127        rostralmiddlefrontal_1
128        rostralmiddlefrontal_2
129        rostralmiddlefrontal_3
130        rostralmiddlefrontal_4
131        rostralmiddlefrontal_5
132        rostralmiddlefrontal_6
133             superiorfrontal_1
134             superiorfrontal_2
135             superiorfrontal_3
136             superiorfrontal_4
137             superiorfrontal_5
138             superiorfrontal_6
139             superiorfrontal_7
140             superiorfrontal_8
141             superiorfrontal_9
142         caudalmiddlefrontal_1
143         caudalmiddlefrontal_2
144         caudalmiddlefrontal_3
145           

/tmp/ipykernel_23993/1691492260.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  labels_BMI=labels[115:len(labels)]


In [132]:
BMI_mean_genes.index = range(116,235)

In [133]:
data=BMI_genes_parcel.get_fdata()

data[np.where(data < 116)]=float("nan")

for i in range(116,235):
    data[np.where(data == i)]=BMI_mean_genes[i]   
    
BMI_genes_img = nib.Nifti1Image(data, BMI_genes_parcel.affine, BMI_genes_parcel.header)
nib.save(BMI_genes_img, '/home/bic/fmorys/Desktop/image.nii')

# OLD

### Abagen order only LH

In [342]:
struct_aba=np.array(['bankssts',
'caudalanteriorcingulate',
'caudalmiddlefrontal',
'cuneus',
'entorhinal',
'fusiform',
'inferiorparietal',
'inferiortemporal',
'isthmuscingulate',
'lateraloccipital',
'lateralorbitofrontal',
'lingual',
'medialorbitofrontal',
'middletemporal',
'parahippocampal',
'paracentral',
'parsopercularis',
'parsorbitalis',
'parstriangularis',
'pericalcarine',
'postcentral',
'posteriorcingulate',
'precentral',
'precuneus',
'rostralanteriorcingulate',
'rostralmiddlefrontal',
'superiorfrontal',
'superiorparietal',
'superiortemporal',
'supramarginal',
'frontalpole',
'temporalpole',
'transversetemporal',
'insula',
'thalamusproper',
'caudate',
'putamen',
'pallidum',
'accumbensarea',
'hippocampus',
'amygdala',
'bankssts',
'caudalanteriorcingulate',
'caudalmiddlefrontal',
'cuneus',
'entorhinal',
'fusiform',
'inferiorparietal',
'inferiortemporal',
'isthmuscingulate',
'lateraloccipital',
'lateralorbitofrontal',
'lingual',
'medialorbitofrontal',
'middletemporal',
'parahippocampal',
'paracentral',
'parsopercularis',
'parsorbitalis',
'parstriangularis',
'pericalcarine',
'postcentral',
'posteriorcingulate',
'precentral',
'precuneus',
'rostralanteriorcingulate',
'rostralmiddlefrontal',
'superiorfrontal',
'superiorparietal',
'superiortemporal',
'supramarginal',
'frontalpole',
'temporalpole',
'transversetemporal',
'insula',
'thalamusproper',
'caudate',
'putamen',
'pallidum',
'accumbensarea',
'hippocampus',
'amygdala',
'brainstem'])


In [10]:
values = np.random.rand(219)
scale = 'scale125'
cammoun = fetch_cammoun2012('fsaverage', verbose=False)[scale]
plot_fsaverage(values, order='LR',lhannot=cammoun.lh,rhannot=cammoun.rh)# doctest: +SKIP

ImportError: Cannot use plot_fsaverage() if pysurfer is not installed. Please install pysurfer and try again.

In [258]:
parcellated_map=[val for sublist in np.ndarray.tolist(mymap_parc) for val in sublist][0:42]
parc_dict=dict(zip(list(labels), parcellated_map))

In [12]:
pip install mayavi

  Using cached mayavi-4.7.4.tar.gz (7.1 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [821 lines of output]
      running bdist_wheel
      running build
      ----------------------------------------------------------------------
      Building TVTK classes... vtkContextDevice2D: Ignoring method: Get/SetViewportRect
      default: [-1318507920, 32750, -1318507920, 32750], range: None
      vtkContextDevice2D: Ignoring method: Get/SetViewportSize
      default: [-1318507920, 32750], range: None
      vtkEventDataForDevice: Ignoring method: Get/SetAction
      default: -1, range: None
      vtkEventDataForDevice: Ignoring method: Get/SetDevice
      default: -1, range: None
      vtkFXAAOptions: Ignoring method: Get/SetDebugOptionValue
      default: 0, range: None
      vtkIncrementalForceLayout: Ignoring method: Get/SetGravityPoint
      default: [200.0, 200.0], ran

Failed to build mayavi
  Running setup.py install for mayavi ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for mayavi did not run successfully.
  │ exit code: 1
  ╰─> [846 lines of output]
      running install
      running build
      ----------------------------------------------------------------------
      Deleting possibly old TVTK classes
      Building TVTK classes... vtkContextDevice2D: Ignoring method: Get/SetViewportRect
      default: [-1973625808, 32739, -1973625808, 32739], range: None
      vtkContextDevice2D: Ignoring method: Get/SetViewportSize
      default: [-1973625808, 32739], range: None
      vtkEventDataForDevice: Ignoring method: Get/SetAction
      default: -1, range: None
      vtkEventDataForDevice: Ignoring method: Get/SetDevice
      default: -1, range: None
      vtkFXAAOptions: Ignoring method: Get/SetDebugOptionValue
      default: 0, range: None
      vtkIncrementalForceLayout: Ignoring method: Get/SetGravityPoint
   

Note: you may need to restart the kernel to use updated packages.


In [260]:
reordered_parc_dict = {k: parc_dict[k] for k in order}